## Model and Data

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
import json
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset
from safetensors.torch import save_file
from peft import AdaLoraConfig, AdaLoraModel, get_peft_model, LoraConfig, TaskType, PeftModel


In [ ]:
def load_and_preprocess_data(train_file, validation_file, tokenizer): 
    
    data_files = {
        'train': train_file,
        'validation': validation_file
    }
    dataset = load_dataset('json', data_files=data_files)
    
    def preprocess_function(examples):
       
        max_length = 32


        inputs = examples['input']
        outputs = [str(o) for o in examples['output']]
        prompts = [f"{inp}\n" for inp in inputs]
       
        full_texts = [prompt + out for prompt, out in zip(prompts, outputs)]

        tokenized_full = tokenizer(full_texts, truncation=True, padding='max_length', max_length=max_length)

        tokenized_prompt = tokenizer(prompts, truncation=True, padding='max_length', max_length=max_length)

        labels = []
        for i in range(len(full_texts)):
     
            prompt_len = len(tokenizer.encode(prompts[i], truncation=True, max_length=max_length))
    
            label = [-100] * prompt_len + tokenized_full['input_ids'][i][prompt_len:]
      
            label = label[:max_length]
        
            if len(label) < max_length:
                label += [-100] * (max_length - len(label))
            labels.append(label)

     
        tokenized_full['labels'] = labels

        return tokenized_full
    

    tokenized_datasets = dataset.map(preprocess_function, batched=True)
    
    tokenized_datasets = tokenized_datasets.remove_columns(['input', 'output', 'instruction'])
    
    return tokenized_datasets


train_file = '/2_arithmetic_operations_100/finetune_pythia_100/finetune_data/train_100.jsonl'
validation_file = '/2_arithmetic_operations_100/finetune_pythia_100/finetune_data/test_100.jsonl'


model_name = 'facebook/opt-6.7b'
tokenizer = AutoTokenizer.from_pretrained(model_name)


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


tokenized_datasets = load_and_preprocess_data(train_file, validation_file, tokenizer)


print(tokenized_datasets['train'][:5])
print(tokenized_datasets['validation'][:5])

train_size = len(tokenized_datasets['train'])
validation_size = len(tokenized_datasets['validation'])



## Lora Loading

In [ ]:
from transformers import AutoModelForCausalLM

model_name = 'facebook/opt-6.7b'
model = AutoModelForCausalLM.from_pretrained(model_name)

for name, module in model.named_modules():
    print(name)

In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained(model_name)


lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,   
    inference_mode=False,          
    r=32,  
    lora_alpha=64,  
    lora_dropout=0,  
    target_modules=[
        'q_proj',  
        'k_proj', 
        'v_proj',  
        'out_proj',  
        'fc1', 
        'fc2' 
    ]
)


model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


In [5]:
# 设置训练参数
training_args = TrainingArguments(
    output_dir='./lora_opt_results/r32a64',    
    num_train_epochs=2,                        
    per_device_train_batch_size=8,            
    warmup_steps=50,                            
    weight_decay=0.01,                         
    logging_dir='./circuit_weighted_lora_logs',   
    logging_steps=10,                              # Log every 10 steps
    save_steps=28,                                
    save_strategy="steps",                       
    save_total_limit=10,                            
    fp16=True,                                     
    gradient_accumulation_steps=4,                 # Gradient accumulation steps
    report_to="none",                             
    learning_rate=3e-4,                            
    
)


## Training

In [6]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer
)


In [ ]:
trainer.train()